# Estimating the cost of quantum experiments in Amazon Braket

### Amazon Braket pricing structure

Amazon Braket provides access to quantum computers, managed simulators that simulate quantum circuits, and managed notebook development environments. There are [two main pricing components](https://aws.amazon.com/braket/pricing/) when using a quantum computer, or quantum processing unit (QPU), on Amazon Braket: a **per-shot** fee and a **per-task** fee.

* A shot is a single execution of a quantum algorithm on a QPU. For example, a shot is a single pass through each stage of a complete quantum circuit on a gate-based QPU from IonQ or Rigetti. When you use a D-Wave quantum annealer, a shot is when you obtain a result sample of a quantum annealing problem. The per-shot pricing depends on the type of QPU used. The per-shot price is not affected by the number or type of gates used in a quantum circuit or the number of variables used in a quantum annealing problem.  


* A task is a sequence of repeated shots based on the same circuit design or annealing problem. You define how many shots you want included in a task when you submit the task to Amazon Braket. Currently per-task pricing is the same across all QPUs.

| Hardware Provider  | QPU family     | Per-task price  | Per-shot price |
|:-------------------|:---------------|:----------------|:----------------|
| D-Wave             | 2000Q          | 0.30000        | 0.00019       |
| D-Wave             | Advantage 1.1  | 0.30000        | 0.00019       |
| D-Wave             | Advantage 4.1  | 0.30000        | 0.00019       |
| IonQ               | IonQ device    | 0.30000        | 0.01000       |
| Rigetti            | Aspen-9        | 0.30000        | 0.00035       |  


### Getting pricing information from Amazon Web Services Price List API

[AWS Price List Service API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/pricing.html) is a centralized and convenient way to programmatically query Amazon Web Services for services, products, and pricing information. The Amazon Web Services Price List Service uses standardized product attributes such as service name and location, and provides prices at the stock keeping unit (SKU) level.  

In this notebook you will use the AWS Price List Service API to obtain all the relevant pricing information for Amazon Braket. You can use the AWS Price List Service to build cost control and scenario planning tools, reconcile billing data, forecast future spend for budgeting purposes, and provide cost benefit analysis that compare your internal workloads with Amazon Web Services.  

You can use ``GetServices`` without a service code to retrieve the service codes for all AWS services, then ``GetServices`` with a service code to retreive the attribute names for that service. For Amazon Braket, you can set ``'AmazonBraket'`` as the atrribute for the service code. After you have the service code and attribute names, you can use ``GetAttributeValues`` to see what values are available for an attribute. ``usagetype`` is the attribute name that contains all the pricing information. With the service code and an attribute name and value, you can use ``GetProducts`` to find specific products that you're interested in, such as the shot and task price for a specific QPU.

In [1]:
import boto3
from pprint import pprint
import json
from braket.aws import AwsDevice, AwsQuantumTask

In [2]:
def get_price_list():

    """ 
    This function creates a dictionary that uses the usage type as key, and the cost as an attribute.
    The dictionary is built by filtering the two billing components in Amazon Braket: task and shot.
    {'Global-Simulators-Task': 0.0,
     'USE1-Task': 0.3,
     'USE1-Task-Shot': 0.01,
     'USW1-Task': 0.3,
     'USW1-Task-Shot': 0.00035,
     'USW2-Task': 0.3,
     'USW2-Task-Shot': 0.00019}
    
    """ 

    # Initialize the client in one of the available endpoints:
    # https://api.pricing.us-east-1.amazonaws.com
    # https://api.pricing.ap-south-1.amazonaws.com
    client = boto3.client('pricing', region_name= 'us-east-1')

    # 'usagetype' contains the pricing information for tasks and shots in Amazon Braket, per region.
    paginator = client.get_paginator('get_attribute_values')
    response_iterator = paginator.paginate(
        ServiceCode='AmazonBraket',
        AttributeName='usagetype',
    )

    # For the list of atributes in usage type, filter the strings ending 
    # by either "Task" or "Shot", and create a list with all of them.
    tasks = []
    for item in response_iterator:
        for i in item['AttributeValues']:
            # This condition is critical to get the right usage types
            if i['Value'][-4:] == 'Task'or i['Value'][-4:] == 'Shot':    
                tasks.append(i['Value']) 

    # Get all products to identify the cost key and its value.
    paginator = client.get_paginator('get_products')
    response_iterator = paginator.paginate(
        ServiceCode='AmazonBraket',
    )

    price_list = {}

    for responses in response_iterator:
        for i in responses['PriceList']:
            # Convert string to json (which creates dict)
            product = json.loads(i)
            #pprint(product)

            # Check if this product is in the selected tasks above
            usage_type = product['product']['attributes']['usagetype']
            if usage_type in tasks:
                ondemand_terms = product['terms']['OnDemand']

                # We could assume there may be only one term and use keys(ondemand_terms)[0]
                # but there may be cases where there are more than one term so better to be safe.
                for terms_id in ondemand_terms:

                    # We could assume there may be only one priceDimension and use keys(terms_id)[0]
                    # but there may be cases where there are more than one term so better to be safe.
                    for j in (ondemand_terms[terms_id]['priceDimensions']):
                        price_list[usage_type] = float(ondemand_terms[terms_id]['priceDimensions'][j]['pricePerUnit']['USD'])

    return price_list

Once you have the price list with shot and task prices for all regions, you can build logic to return the right value depending on the device and its location. 

The next function takes the device as a parameter, and returns the task and shot price for it. Currently the map between region and device is 1:1 (no region information in the arn), but that can be easily modified if it changes in the future.

The advantage of using 2 different functions is that you don't need to execute ``get_price_list`` everytime, but only when prices are updated or you want a refresh with the latest information for other reason. It will also return the pricing information for new devices as they get added.

In [3]:
def get_price(device=None):

    """    
    Function to match dict entries with the right ARN, and assign them to price_per_task and price_per_shot.
    For example: If device is ionq, pass USE1-Task and USE1-Task-Shot values. 
    Since one atribute can match >1 device, we need to specify each case:
    Note: Current schema will stop working if an existing device model is added to a new region.
    
    ----------------------------------------------------------------------------------------------
     Attribute                 Device
    ----------------------------------------------------------------------------------------------
     Global-Simulators-Task    AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")
     Global-Simulators-Task    AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/tn1")
     Global-Simulators-Task    AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/dm1")
     USE1-Task[-Shot]          AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
     USW1-Task[-Shot]          AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-9")
     USW2-Task[-Shot]          AwsDevice("arn:aws:braket:::device/qpu/d-wave/Advantage_system1")
     USW2-Task[-Shot]          AwsDevice("arn:aws:braket:::device/qpu/d-wave/Advantage_system4")
     USW2-Task[-Shot]          AwsDevice("arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6")
    ----------------------------------------------------------------------------------------------
    
    """
    price_list = get_price_list()
    
    arn = device._arn
    if 'simulator' in arn:
        price_per_task = price_list['Global-Simulators-Task']
    if 'ionq' in arn:
        price_per_task = price_list['USE1-Task']
        price_per_shot = price_list['USE1-Task-Shot']
    if 'rigetti' in arn:
        price_per_task = price_list['USW1-Task']
        price_per_shot = price_list['USW1-Task-Shot']
    if 'd-wave' in arn:
        price_per_task = price_list['USW2-Task']
        price_per_shot = price_list['USW2-Task-Shot']
        
    return {'task': price_per_task, 'shot': price_per_shot}

### Amazon Braket QPU Cost Estimator

After the preliminary steps, you are ready to calculate the overall cost of your experiment or simulation. The following cells provide a function to estimate the cost of an experiment using a quantum computer, and a set of cost estimate best practices to follow when submitting calculations. 

<div class="alert alert-block alert-info">
    <b>Note:</b> Besides the cost associated with the use of the QPU, there are also other costs for the services used to store results (Amazon S3), run the notebook itself, and other optional services like the setting up of alarms with AWS CloudWatch or Amazon EventBridge. You will be billed separately for the use of each of these capabilities. The cost estimate function provided below does not account for the cost of these services. 
</div>

In [4]:
def estimate_cost(device=None, tasks=1, shots_per_task = 1000):
    
    """
    Function to return the estimate cost of QPU simulation. 

    Input:      device = AwsDevice("arn:aws:braket:::device/[quantum-simulator|qpu]/VENDOR/MODEL)
                tasks = maximum number of tasks allowed in a variational calculation (user defined parameter)
                shots_per_task = number of shots per task (user defined parameter)   
    
    Example:    estimate_cost\
                    (device = 'AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-9")',\
                    tasks = 1, shots_per_task = 1000)
    
    Example output: 
        --------------------------------------------------------
        AWS Device:       Aspen-9
        Number of tasks:  1
        Shots per task:   1000
        Price per task:   0.3
        Price per shot:   $0.00035

        Cost estimate:    $0.6499999999999999
        --------------------------------------------------------      
    
    Note: 
        This function calculates the cost from the number of tasks and shots. 
        It does not include the cost associated with other AWS services (Braket notebooks, S3, etc.)
        
    """
    # Function 'get_price' returns 2 values: task and shot   
    price_per_task = get_price(device)['task']
    price_per_shot = get_price(device)['shot']
    
    # We can also obtain the shot price using Braket SDK, but task price doesn't exist there.
    # price_per_shot = device.properties.service.deviceCost.price 

    price = tasks * price_per_task \
          + tasks * shots_per_task * price_per_shot  
    print('--------------------------------------------------------')
    print('AWS Device:       {}'.format(device.name))    
    print('Number of tasks:  {}'.format(tasks))
    print('Shots per task:   {}'.format(shots_per_task))
    print('Price per task:   ${}'.format(price_per_task))
    print('Price per shot:   ${}\n'.format(price_per_shot))
    print('Cost estimate:    ${}'.format(price)) 
    print('--------------------------------------------------------\n\n')
   

### Calculating costs
Estimate the cost of your circuit **before** submitting to the QPU.

In [5]:
# Estimate the cost of a circuit on the D-Wave device
dev = AwsDevice("arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6")
estimate_cost(device = dev, tasks = 10, shots_per_task = 10000)

--------------------------------------------------------
AWS Device:       DW_2000Q_6
Number of tasks:  10
Shots per task:   10000
Price per task:   $0.3
Price per shot:   $0.00019

Cost estimate:    $22.0
--------------------------------------------------------


